In [2]:

from time import time
import numpy as np
from tst.domutil.util import *
from tst.domutil.test import *
from tst.utils_db import *


from copy import copy
from scipy.sparse import *
from scipy.spatial.distance import *


def Dnorm(hc1, hc2, hcboth):
    log1 =  np.log10( hc1 + 1) 
    log2 =  np.log10( hc1 + 1) 
    log3 =  np.log10( hcboth + 1) 
    norm = (log1 + log2) / 2 - log3 
    return norm

reset_database_connection()

# import sys
reload(sys.modules["tst.domutil.util"])
from tst.domutil.util import *

import sys
reload(sys.modules["tst.domutil.test"])
from tst.domutil.test import *

import sys
reload(sys.modules["tst.utils_db"])
from tst.utils_db import *

# %%time
reset_database_connection()

sDB = seqDB.objects.get(name = 'CATH')
# sDB.sequence_set.count()
seq = sDB.sequence_set.first()

# seqset_all = sDB.sequence_set.all().prefetch_related('hmmprofile_set').filter(hit4hmm2hsp__bitscore__gte=30)

# seq.HMMprofile
# seq.hmmprofile
# dir(seq)
# seq.hmmprofile_set.count()

sDB = seqDB.objects.get(name = 'CATH')
# sDB.sequence_set.count()
seq = sDB.sequence_set.first()

# seqset_all = sDB.sequence_set.all().prefetch_related('hmmprofile_set').filter(hit4hmm2hsp__bitscore__gte=30)

# seq.HMMprofile
# seq.hmmprofile
# dir(seq)
# seq.hmmprofile_set.count()


settings.USE_MODELLER
settings.USE_MODELLER


In [2]:
seqs = sDB.sequence_set.all().defer('hmmprofile__text')
matObj = ctmat( HMMprofile.objects, seqs = seqs, letter = 'H')

classification.objects.filter(pk = 310499).values_list("classification")
# node = classification.objects.filter(pk__gte = 293169)[0]
hmms = HMMprofile.objects.all()[:100]

In [10]:
hmms.__dict__
hmms.values_list('id','hits').model == HMMprofile

True

In [8]:
def hitlist_compare(node1__id = 309754, node2__id = 310524, sDB = None, **kwargs):

    qset1 = classification.objects.filter(id = node1__id)
    qset2 = classification.objects.filter(id = node2__id)

    node1 = qset1[0]
    node2 = qset2[0]


    rv_field = reverse_field[ node1.level.letter ]
    sDB_field  = rv_field + "__seqDB"
    text_field = rv_field.replace('__hits','__text')
    rvid_field = rv_field.replace('__hits','__hit4hmm2hsp__id')
    # qset1 = classification.objects.filter(id = self.node1.id)


    rv_field = reverse_field[node2.level.letter]
    sDB_field  = rv_field + "__seqDB"
    text_field = rv_field.replace('__hits','__text')
    rvid_field = rv_field.replace('__hits','__hit4hmm2hsp__id')

    # qset2 = classification.objects.filter(id = self.node2.id)

    # hitlist1 = qset1.defer(text_field).values_list(rvid_field, flat = True) 
    # hitlist2 = qset2.defer(text_field).values_list(rvid_field, flat = True) 

    # hits1_ids = {dict_hit2tid[ i ]: i for i in hitlist1 if i }
    # hits2_ids = {dict_hit2tid[ i ]: i for i in hitlist2 if i }

    hitlist1 = qset1.defer(text_field).values_list(rvid_field, sDB_field) 
    hitlist2 = qset2.defer(text_field).values_list(rvid_field, sDB_field) 
    sDB_id = sDB.id
    hits1_ids = {dict_hit2tid[ i ]: i for i,j in hitlist1 if i and j==sDB_id}
    hits2_ids = {dict_hit2tid[ i ]: i for i,j in hitlist2 if i and j==sDB_id}

    # hits2_ids = {x.target.id: x for x in hits2dict.values()}
    inter_ids = set( hits1_ids ) & set( hits2_ids )

    hits1_obj = hit4hmm2hsp.objects.in_bulk( [hits1_ids[i] for i in inter_ids] )
    hits2_obj = hit4hmm2hsp.objects.in_bulk( [hits2_ids[i] for i in inter_ids] )

    print len(inter_ids)
    qset = [ 
    hitP_obj(id = i,
        hit1 = hit1,
        hit2 = hit2) for i,hit1,hit2 in 
        izip(
            inter_ids,hits1_obj.values(),hits2_obj.values()
        )

    ]


    if isinstance(qset, list):
        qset = fake__query_set(qset)
        # qset.__str__ = lambda self:'%2.1f  %2.1f' % (self.hit1.bitscore, self.hit2.bitscore)
    print "[debug]:length of qset is",len(qset)

    return qset

sDB = seqDB.objects.get(name ='CATH')
len(hitlist_compare(sDB = sDB))

45
[debug]:length of qset is 0


0

In [ ]:
from tst.views import *
sDB = seqDB.objects.get(name ='CATH')


In [74]:
#####
### Symptom:  fake__query_set() produces an object of length 0.
#####

# qset = hitlist_compare(sDB = sDB)
qset = fake__query_set(hitlist_compare(sDB = sDB))

fields  = [
    'hit1__bitscore',
    'hit2__bitscore',
    'id',
    # 'superfamily',
    ]

fields  = [
    'hit1__bitscore',
    'hit2__bitscore',
    'hit1__bitscore',
    # 'superfamily',
    ]
it = qset.values_list(*fields)
# print next(it)
# print next(it)
print list(it)

45
[(21.0, 26.3, 21.0), (21.0, 41.0, 21.0), (214.0, 35.4, 214.0), (210.9, 35.4, 210.9), (209.6, 32.7, 209.6), (198.5, 32.7, 198.5), (192.7, 31.1, 192.7), (190.1, 28.3, 190.1), (180.5, 28.6, 180.5), (180.5, 29.0, 180.5), (179.3, 28.8, 179.3), (176.9, 28.6, 176.9), (173.3, 27.2, 173.3), (169.9, 26.3, 169.9), (169.7, 26.2, 169.7), (169.4, 26.2, 169.4), (169.0, 25.6, 169.0), (167.9, 26.2, 167.9), (165.7, 26.2, 165.7), (165.7, 25.5, 165.7), (163.4, 25.5, 163.4), (162.5, 25.4, 162.5), (161.2, 25.5, 161.2), (160.5, 24.4, 160.5), (158.9, 24.5, 158.9), (156.1, 23.0, 156.1), (155.2, 21.9, 155.2), (154.6, 23.3, 154.6), (146.1, 21.8, 146.1), (142.3, 21.8, 142.3), (138.5, 22.1, 138.5), (134.5, 22.2, 134.5), (134.5, 22.2, 134.5), (134.5, 22.1, 134.5), (134.5, 22.1, 134.5), (132.1, 22.1, 132.1), (129.2, 22.1, 129.2), (129.2, 20.1, 129.2), (127.5, 22.0, 127.5), (26.7, 171.2, 26.7), (21.9, 21.5, 21.9), (120.1, 20.4, 120.1), (26.6, 20.2, 26.6), (21.9, 21.6, 21.9), (127.5, 113.5, 127.5)]


In [ ]:
help(hitlist_compare)

In [1]:
from tst.views import *
qset = classification.objects.in_bulk([309877,310384]).values()
qset = fake__query_set(qset)
# qset.data
qset

[<classification: 3.40.50.20.0>, <classification: 3.30.9.10.0>]

In [19]:
init_data()['node1'].dft

0

In [13]:
hit4cath2cath.objects.get(node1=309877,node2=310384)._meta.model

tst.models.hit4cath2cath

In [17]:
##########
## test ##
##########

reset_database_connection()

def init_data():
    node1 = classification.objects.get(id=309877) ### supfam:3.30.9.10.0
    node2 = classification.objects.get(id=310384) ### supfam:3.40.50.20.0
    sDB = seqDB.objects.get(name = 'CATH')
    exp = 68
    return( {'node1':node1,'node2':node2,'sDB':sDB, 'exp':exp})


def assert_expr(act, expr, behave = None, **kwargs):
#     expr0 = expr[:]
    fullexpr = ' act ' + expr
    basemsg = "Expecting:%s, Actual:%s " % (expr,act)
    success = eval(fullexpr,locals(),kwargs)
    assert( success ),"[FAILED]:" + basemsg
    print "[PASSED]:" + basemsg


def test__hitlist_compare():
#     locals().update(init_data())
    func_name = 'hitlist_compare'
    func = eval(func_name)
    print "[testing '%s']" %func_name
    data = init_data()
        
    hitPlist = func( data["node1"].id, data["node2"].id, data["sDB"] )

    
    behave = "Testing length of output list"
    exp = data["exp"]
    assert_expr( len(hitPlist), " == exp ", **data )
#     assert act == data["exp"],'[failed] %s exp:%s, act:%s ' 

In [208]:
test__hitlist_compare()

[testing 'hitlist_compare']
68
[PASSED]:Expecting: == exp , Actual:68 


In [207]:
# %timeit hitlist_compare(sDB = sDB)
%load_ext line_profiler
def test():
    for i in range(20):
        hitlist_compare(sDB = sDB)

%lprun -f hitlist_compare test()

The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler
45
45
45
45
45
45
45
45
45
45
45
45
45
45
45
45
45
45
45
45


In [206]:
def hitlist_compare(node1__id = 309754, node2__id = 310524, sDB = None, **kwargs):
    sDB_id = sDB.id

    qset1 = classification.objects.filter(id = node1__id)
    qset2 = classification.objects.filter(id = node2__id)

    node1 = qset1[0]
    node2 = qset2[0]


    rv_field = reverse_field[ node1.level.letter ]
    sDB_field  = rv_field + "__seqDB"
    text_field = rv_field.replace('__hits','__text')
    rvid_field = rv_field.replace('__hits','__hit4hmm2hsp__id')

    fs = [rv_field, rvid_field, sDB_field]
    hitlist1 = qset1.values_list(*fs).distinct() 

    
    rv_field = reverse_field[ node2.level.letter ]
    sDB_field  = rv_field + "__seqDB"
    text_field = rv_field.replace('__hits','__text')
    rvid_field = rv_field.replace('__hits','__hit4hmm2hsp__id')
    
    fs = [rv_field, rvid_field, sDB_field]
    hitlist2 = qset2.values_list(*fs).distinct() 

#     hitlist2 = filter(ffunc, hitlist2)
#     hits1_ids = {x[0]: x[1] for x in hitlist1}
#     hits2_ids = {x[0]: x[1] for x in hitlist2}

    
    hits1_ids = {seqid:hitid for seqid,hitid,sid in hitlist1 if sid == sDB_id}
    hits2_ids = {seqid:hitid for seqid,hitid,sid in hitlist2 if sid == sDB_id}
    inter_ids = set(hits1_ids) & set(hits2_ids)

    hits1_obj = hit4hmm2hsp.objects.in_bulk( [hits1_ids[i] for i in inter_ids] )
    hits2_obj = hit4hmm2hsp.objects.in_bulk( [hits2_ids[i] for i in inter_ids] )

    print len(inter_ids)
    qset = [ 
    hitP_obj(
        id = i,
        acc = hit1.target.acc,
        hit1 = hit1,
        hit2 = hit2) for i,hit1,hit2 in 
        izip(
            inter_ids,hits1_obj.values(),hits2_obj.values()
        )

    ]


    return qset


In [82]:
n1 = classification.objects.get(id=309877)
n2 = classification.objects.get(id=310384)
print n1
print n2

3.30.9.10.0
3.40.50.20.0
